In [2]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [3]:
### TEST FIXER LA VERSION 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [4]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Retrieving notices: done
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - compilers=1.10
    - go=1.24.5
    - jq=1.8.1
    - make=4.4.1
    - zlib=1.3.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _go_select-2.2.0           |            nocgo           5 KB  conda-forge
    binutils-2.43              |       h4852527_5          34 KB  conda-forge
    binutils_linux-64-2.43     |       h4852527_4          35 KB  conda-forge
    c-compiler-1.10.0          |       h2b85faf_0           7 KB  conda-forge
    ca-certificates-2025.7.14  |       hbd8a1cb_0         152 KB  conda-forge

In [5]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [6]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [7]:
cd obitools4
make
cd ..

mkdir -p build
Update version.go : pkg/obioptions/version.go to Release 4.4.0 (996ec69)
- Building obitool obiannotate...Done.
- Building obitool obiclean...Done.
- Building obitool obicleandb...Done.
- Building obitool obicomplement...Done.
- Building obitool obiconsensus...Done.
- Building obitool obiconvert...Done.
- Building obitool obicount...Done.
- Building obitool obicsv...Done.
- Building obitool obidemerge...Done.
- Building obitool obidistribute...Done.
- Building obitool obigrep...Done.
- Building obitool obijoin...Done.
- Building obitool obikmermatch...Done.
- Building obitool obikmersimcount...Done.
- Building obitool obilandmark...Done.
- Building obitool obimatrix...Done.
- Building obitool obimicrosat...Done.
- Building obitool obimultiplex...Done.
- Building obitool obipairing...Done.
- Building obitool obipcr...Done.
- Building obitool obireffamidx...Done.
- Building obitool obirefidx...Done.
- Building obitool obiscript...Done.
- Building obitool obisplit...Done.
-

In [ ]:
## FORMATAGE DES DONNES / ESPACES RETIRE
shopt -s nullglob
for f in galaxy_inputs/inputfile/*.{fastq,fasta}; do
  [ -e "$f" ] || continue
  dir=$(dirname -- "$f")
  base=$(basename -- "$f")
  # on enlève les espaces
  newbase=${base// /_}
  newpath="$dir/$newbase"
  [ "$newpath" = "$f" ] && continue
  echo "Renomme: $f -> $newpath"
  mv -- "$f" "$newpath"
done

In [ ]:
jq '.inputfile[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_fixed.json
json_path="galaxy_inputs/galaxy_inputs.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')

echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"
format=""


Input file path: galaxy_inputs/inputfile/NGSfilterondata22anddata10.fastqsanger
Nom de fichier (sans chemin, sans extension) : NGSfilterondata22anddata10
Extension détectée : fastqsanger


In [24]:
#### Variables
# category attribute=""
attrib=$(jq -r '.attrib' $json_path)

# merg=""   
merge=$(jq -r '.merge' $json_path)
####

# no singleton
single=$(jq -r '.noXsingleton' $json_path)

echo "$input_file"
echo "$single"

galaxy_inputs/inputfile/NGSfilterondata22anddata10.fastqsanger
true


In [ ]:
if [[ -n "$attrib"  && "$attrib" != "null" ]]; then
    attrib="-c $attrib"
    echo "variable attrib présente: $attrib"
else 
    attrib=""
fi

if [[ -n "$merge"  && "$merge" != "null"  ]]; then
    merge="-m $merge"
    echo "variable def présente : $merge"
else 
    merge=""
fi

if [[ "$single" == "true"  ]]; then
    single="--no-singleton"
    echo "variable single TRUE"
else 
    single=""
fi

#Chemin paired end 

pairedend_file=$(jq -r '.pairedend[0].path' galaxy_inputs/galaxy_inputs.json)

if [[ -n "$pairedend_file"]]; then
    pairedend_file="--paired-with $pairedend_file"
    echo "variable pairedend présente: $pairedend"
fi

variable attrib présente: -c sample
variable def présente : -m merge
variable single TRUE
--no-singleton


In [26]:
./obitools4/build/obiuniq $attrib $merge $input_file $single $pairedend --out $output_file --no-order

INFO[0000] No singleton option set                      
INFO[0000] Number of workers set 8                      
INFO[0000] Found 1 files to process                     
INFO[0000] /data/jwd06/pulsar_staging/86617778/inputs/dataset_14f0e932-8896-41da-b63f-2c999e4d6335.dat mime type: text/fastq 
INFO[0000] Running dereplication on disk with 100 chunks 
INFO[0000] Removing sigletons from the output           
INFO[0000] Starting data splitting                      
- Splitting data set (42339/-, 5925 it/s) [7s] 

INFO[0007] Data splitted over 100 batches               
INFO[0007] End of the data splitting                    
INFO[0007] On output use JSON headers                   
INFO[0007] /tmp/obiseq_chunks_2350436625/chunk_0.fastx mime type: text/fasta 
INFO[0007] Start processing of batch 0/100 : 95 sequences 
INFO[0007] /tmp/obiseq_chunks_2350436625/chunk_1.fastx mime type: text/fasta 
INFO[0007] Start processing of batch 1/100 : 55 sequences 
INFO[0007] /tmp/obiseq_chunks_2350436